In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
np.random.seed(42)

# 1. Data processing and Model Defination 

In [3]:
X_train = pd.read_csv("train_X_cat.csv",index_col=0)
Y_train = pd.read_csv("train_y_sales.csv",index_col=0)

In [4]:
Model_1 = DecisionTreeRegressor()

# 2. Data Pipelining with GridSearch 

In [9]:
pca=PCA()
RG = Model_1
pipe = Pipeline(steps=[("pca",pca),("RG",RG)])

param_grid = {
    "pca__n_components": [5, 10, 30, 45, 60, 100 ,150 ,170],
    "RG__ccp_alpha":[0,0.001,0.005,0.1,0.5,2,8]
}
score_metric = 'neg_mean_squared_error'
search = GridSearchCV(pipe,param_grid,
                      n_jobs=-1,
                      cv=10,
                      scoring=score_metric)
search.fit(X_train,Y_train)

D:\Download\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
70 fits failed out of a total of 560.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Download\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Download\Anaconda\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "D:\Download\Anaconda\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "D:\Download\Anaconda\lib\site-package

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('pca', PCA()),
                                       ('RG', DecisionTreeRegressor())]),
             n_jobs=-1,
             param_grid={'RG__ccp_alpha': [0, 0.001, 0.005, 0.1, 0.5, 2, 8],
                         'pca__n_components': [5, 10, 30, 45, 60, 100, 150,
                                               170]},
             scoring='neg_mean_squared_error')

In [10]:
df = pd.DataFrame(search.cv_results_)
df = df.sort_values(by=['rank_test_score'])
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_RG__ccp_alpha,param_pca__n_components,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
50,0.095058,0.030376,0.006091,0.004746,8,30,"{'RG__ccp_alpha': 8, 'pca__n_components': 30}",-571.218376,-343.370134,-2089.927483,-619.297790,-1274.450389,-2662.517570,-1340.363174,-2503.515170,-1474.888912,-640.041411,-1351.959041,792.459664,1
16,0.040042,0.015067,0.003411,0.000834,0.005,5,"{'RG__ccp_alpha': 0.005, 'pca__n_components': 5}",-831.313512,-594.827704,-1548.911225,-950.497002,-2957.714063,-1745.350049,-1318.330227,-3102.132238,-446.287428,-499.554136,-1399.491758,914.656545,2
35,0.112573,0.020212,0.004807,0.003826,0.5,45,"{'RG__ccp_alpha': 0.5, 'pca__n_components': 45}",-1331.095760,-426.858784,-1409.879699,-582.282713,-1371.383110,-2593.060713,-1577.920342,-2500.833713,-1822.651766,-594.317687,-1421.028429,714.951093,3
0,0.030115,0.002568,0.004078,0.001594,0,5,"{'RG__ccp_alpha': 0, 'pca__n_components': 5}",-799.943038,-499.474684,-1515.613924,-2259.727848,-3036.120253,-1414.879747,-1252.477707,-2547.668790,-460.955414,-486.152866,-1427.301427,874.585488,4
24,0.036725,0.011292,0.003696,0.001242,0.1,5,"{'RG__ccp_alpha': 0.1, 'pca__n_components': 5}",-914.711829,-521.636439,-1324.243158,-2325.370292,-3631.022294,-1486.146169,-1048.609018,-2440.938500,-455.836891,-636.478269,-1478.499286,973.333174,5
10,0.087681,0.011908,0.004274,0.001089,0.001,30,"{'RG__ccp_alpha': 0.001, 'pca__n_components': 30}",-586.707507,-415.824482,-1505.111950,-760.336966,-1542.032469,-2265.453180,-980.783008,-3835.692684,-2291.807969,-627.211541,-1481.096176,1012.647282,6
40,0.034055,0.006702,0.003110,0.000775,2,5,"{'RG__ccp_alpha': 2, 'pca__n_components': 5}",-1059.219714,-376.784587,-1540.104333,-2372.345943,-2955.310644,-1718.323545,-949.926543,-2866.495682,-474.629932,-558.213272,-1487.135420,921.448450,7
34,0.075081,0.014146,0.003087,0.000758,0.5,30,"{'RG__ccp_alpha': 0.5, 'pca__n_components': 30}",-518.976277,-433.311777,-2401.841111,-1231.166685,-1512.247554,-2411.262187,-1299.502081,-2817.921737,-1886.933874,-604.026953,-1511.719024,808.986904,8
25,0.045669,0.011689,0.003058,0.000516,0.1,10,"{'RG__ccp_alpha': 0.1, 'pca__n_components': 10}",-970.706998,-522.057263,-2058.648661,-676.649837,-2294.999096,-2305.837800,-1427.103842,-3325.985773,-1072.248761,-520.367193,-1517.460522,893.703294,9
13,0.216892,0.022770,0.003961,0.001593,0.001,100,"{'RG__ccp_alpha': 0.001, 'pca__n_components': ...",-1729.272548,-562.356846,-1654.049721,-1162.777922,-1717.457257,-2005.723627,-1009.707215,-3062.274981,-1847.425039,-801.364985,-1555.241014,680.558899,10


# 3. Best Hyperparameter

In [11]:
parms = pd.DataFrame(df['params'])
for i in parms.iloc[0]:
    print(i)

{'RG__ccp_alpha': 8, 'pca__n_components': 30}


# 4. Evaluation of the Model with K-Fold for Original Data


In [8]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE

# Clean and preprocess data
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
Y_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(X_train.mean(), inplace=True)
Y_train.fillna(Y_train.mean(), inplace=True)

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Extract the best hyperparameters
best_pca = parms.iloc[0][0]['pca__n_components']
best_ccp_alpha = parms.iloc[0][0]['RG__ccp_alpha']

# Prepare the PCA and the Decision Tree Regressor with the best parameters
pca = PCA(n_components=best_pca)
Model_2 = DecisionTreeRegressor(ccp_alpha=best_ccp_alpha)

# Prepare the k-fold cross-validation configuration
k = 10  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=132)

# Lists to store the MSE for each fold
mse_scaled_values = []
mse_true_values = []

for train_index, test_index in kf.split(X_train_scaled):
    # Split the data into training and validation folds
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[test_index]
    Y_train_fold, Y_val_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    
    # Apply PCA transformation
    PX_train_fold = pca.fit_transform(X_train_fold)
    PX_val_fold = pca.transform(X_val_fold)
    
    # Fit the model on the training fold
    Model_2.fit(PX_train_fold, Y_train_fold)
    
    # Predict on the validation fold and calculate the MSE
    Y_val_fold_pred_scaled = Model_2.predict(PX_val_fold)
    mse_fold_scaled = MSE(Y_val_fold, Y_val_fold_pred_scaled)
    mse_scaled_values.append(mse_fold_scaled)
    
    # Note: If your original data (Y values) were transformed (e.g., log-transformed) and you want to 
    # calculate MSE on the original scale, apply the reverse transformation to both predictions and actual values.
    # Example for log transformation:
    # Y_val_fold_pred = np.exp(Y_val_fold_pred_scaled)
    # Y_val_fold_true = np.exp(Y_val_fold)
    # mse_fold_true = MSE(Y_val_fold_true, Y_val_fold_pred)
    # mse_true_values.append(mse_fold_true)

# Calculate the mean MSE across all folds
mean_mse_scaled = np.mean(mse_scaled_values)
# mean_mse_true = np.mean(mse_true_values)  # Uncomment if the true values calculation is needed

print(f'Mean MSE for scaled values across all folds: {mean_mse_scaled}')
# print(f'Mean MSE for true values across all folds: {mean_mse_true}')  # Uncomment if the true values calculation is needed


C:\Users\1\AppData\Local\Temp\ipykernel_8696\3593803019.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_pca = parms.iloc[0][0]['pca__n_components']
C:\Users\1\AppData\Local\Temp\ipykernel_8696\3593803019.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_ccp_alpha = parms.iloc[0][0]['RG__ccp_alpha']


Mean MSE for scaled values across all folds: 2450.3816584515043


# 5. Evaluation of the Model with K-Fold for Log(Sales) Data



In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

# Assuming best_ccp_alpha and best_n_est have been previously defined
Model_3 = RandomForestRegressor(ccp_alpha=2, n_estimators=5)

# Load the target variable, which is the log-transformed sales, from a CSV file
OY_train = pd.read_csv("train_y_logsales.csv", index_col=0)
# Assuming X_train is already defined and holds the feature data

# Prepare the cross-validation procedure
cv = KFold(n_splits=10, shuffle=True, random_state=132)

# Collect the MSE for each fold
mse_values = []
true_vs_predicted_mse_values = []

for train_ix, test_ix in cv.split(X_train):
    # Split the data
    X_train_fold, X_test_fold = X_train.iloc[train_ix], X_train.iloc[test_ix]
    Y_train_fold, Y_test_fold = OY_train.iloc[train_ix], OY_train.iloc[test_ix]

    # Fit the model on the training data
    Model_3.fit(X_train_fold, Y_train_fold.values.ravel())

    # Predict on the test data and calculate the MSE
    mse_value = mean_squared_error(Y_test_fold, Model_3.predict(X_test_fold))
    mse_values.append(mse_value)
    
    # Predict and transform back from log scale
    predicted_sales = np.exp(Model_3.predict(X_test_fold))
    true_sales = np.exp(Y_test_fold.values.ravel())

    # Calculate the MSE between the true and predicted sales
    final_mse = mean_squared_error(true_sales, predicted_sales)
    true_vs_predicted_mse_values.append(final_mse)

# Calculate the mean MSE across all folds
mean_mse = np.mean(mse_values)
mean_true_vs_predicted_mse = np.mean(true_vs_predicted_mse_values)

print(f'Mean MSE across all folds (scaled values): {mean_mse}')
print(f'Mean MSE between true and predicted sales across all folds: {mean_true_vs_predicted_mse}')



Mean MSE across all folds (scaled values): 2.2397262342552158
Mean MSE between true and predicted sales across all folds: 7172.9505386371575
